<a href="https://colab.research.google.com/github/steeve85ok/SIM_YAC_ALMEIDA_UPSE2023/blob/main/yaokokokexpimp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Solucion Matriz Metodo explicito e implicito

In [ ]:
# Revision d metodos explicito e implicito
import numpy as np #vector
import math

In [ ]:
L=10000 # longitud de reservorio
dx=2500 # separación de nodos
Tiempo= 100000 # numero de días (division a dt se tiene numero de iteraciones)
DT=1 # intervalo de tiempo en dias
K=50 #Permeabilidad MD
por=0.2 #Porosidad u
vis=1 # viscocidad (centipoise)
Beta=1 # factor volumetrico (se asume constante)
CompTotal=1**-6 #compresibilidad total psi-1
Area=200000 #ft**2 (area del reseervorio)



In [ ]:
Alpha=K/(vis*por*CompTotal) # da la forma de ecuación difusividad d2p/dx2=(1/alpha)dp/dt, alpha ya no es uno
Alpha
Eta=Alpha*DT/(dx**2) # para hallar estabilidad varia dx (100 a 2500), DT (100 a 1); oj comp 1**-6  ; si alpha es uno simplemente dx es doble DT
Eta

4e-05

In [ ]:
#Esquema Explícito Pi(**)n+1=Pi(**)n + alphaDT/dx**2 [Pi-1(**)n - 2Pi(**)n + Pi+1(**)n], considerar Eta=alphaDT/dx2, Estable: Eta menor 0.5
def Presdt(P0,P1,P2,Eta): # (funcion esquema eplicito usando Eta en lugar DT y dx)
  Pdt=P1+Eta*(P0-2*P1+P2)
  return(Pdt)

Pexplicita=np.zeros((Tiempo//DT,L//dx)) # matriz que va a guardar todos los resultados. Filas N: Iteraciones, Columnas m: numero de Nodos


In [ ]:
# Condiciones Iniciales esquema explicito

Pyac=1000 # presion de yacimiento psi # se da valores al vector de presiones inciales
Piz=2000 # presion frontera izquierda psi
Pder=0 # presion frontera derecha psi (todo flow)

Pin=np.ones(L//dx) #vector de presiones iniciales (es cantidad de nodos y los datos están con unos)
Pres=np.zeros(L//dx) #vector de resultados
Pin=Pin*Pyac #valores del vector incial como presion de yacimiento (tiempo=0)
#Atribuir valores al vector d condiciones iniciales
Pin[0]=Piz #primera celda (0) del vector Pin será Pizquierda (frontera izquierda)
Pin[len(Pin)-1]=Pder #ultima celda del vector menos 1 es frontera derecha (en python)

Pin

array([2000., 1000., 1000.,    0.])

In [ ]:
Pin[len(Pin)-1]

0.0

In [ ]:
t=DT
Pexplicita[0]=Pin           # en la matriz Pexplicita, la presion de la primera fila será la presión incial
while t<Tiempo:                  # Se empieza calculos con ciclo en base al tiempo, llenarán la fila 2,3,... (se tendran resultados con una iteracion menos)
  for i in range(0,len(Pin)-1):  # creacion del ciclo de espacio (columnas: (0)es primer valor, (Pin)-1 es ultimo valor)
    if i==0:                    # i==0 representa la posición del nodo en todos los puntos de la frontera izquierda
      Pres[i]=Pin[0]            #Pin[i]=Piz definido en condiciones iniciales, (con vector de resultados Pres se llenan  todos los valores de la columna en i =0, frontera izq)
    elif i==len(Pin)-1:         # posición de i en frontera derecha
      Pres[i]=Pin[len(Pin)-1]   #Pin[len(Pin)-1]=Pder definido en condiciones iniciales, (así se llenan todos los valores de la columna final i = num nodos menos 1, frontera der)
    else:                       # en todos los demás puntos  se llamará a la funcion denominada Presdt para las presiones futuras
      Pres[i]=Presdt(Pin[i-1],Pin[i],Pin[i+1],Eta)  # triangulo de posicion i, i-1, i+1, con relacion a valores del vector anterior que sería la presion de i-1, presion de i, presion de i+1; factor Eta (dif a 1); con i da cambio nodo por nodo
  Pexplicita[t//DT]=Pres         # Pres vector pone valores en la siguiente fila de la matriz (sale del condicional)
  t=t+DT                        # siguiente ciclo
  Pin=Pres                      # vector inicial recopila al vector de resultads anterior, y llena los triangulos sucesivamente

In [ ]:
Pexplicita                    # Resultados de la matriz seteada

array([[2000.        , 1000.        , 1000.        ,    0.        ],
       [2000.        , 1000.04      ,  999.96      ,    0.        ],
       [2000.        , 1000.0799952 ,  999.9200064 ,    0.        ],
       ...,
       [2000.        , 1333.33146859,  666.66889757,    0.        ],
       [2000.        , 1333.33146883,  666.66889732,    0.        ],
       [2000.        , 1333.33146907,  666.66889706,    0.        ]])

In [ ]:
Pexplicita.shape

(100000, 4)

In [ ]:
# Esquema Implícito

In [ ]:
# Solucion explicita con sistema de ecuaciones 1:41 del video
# Matriz tridiagonal - Eta, 1+2Eta, -Eta
# Matriz cuadrada NxN,  Tamaño depende de numero de triangulos, cada uno es ecuacion, num incogn es num ecuaciones, lineal
# num ecuaciones N = #nodos-2
# indices i,J de matriz, diagonal si i=J valor numérico del término es 1+2Eta
# índice J-1 ó J+1 corresponden a valor numérico -Eta


In [ ]:
Pimplicita= np.zeros((Tiempo//DT,L//dx))    #  matriz implicita de rsutados (diferente a matriz donde se resolvera las presiones futura o Matriz de resolucion)
Pin=np.ones(L//dx) #vector de presiones iniciales (es cantidad de nodos)
Pres=np.zeros(L//dx) #vector de resultados de ceros
Pin=Pin*Pyac #valores del vector incial como presion de yacimiento (tiempo=0)
Pin[0]=Piz # #Atribuir valores al vector d condiciones iniciales, primera celda (0) del vector Pin será Pizquierda (frontera izquierda)
Pin[len(Pin)-1]=Pder
N=L//dx   # Numero de nodos= long yac / dx
Matriz_resolucion=np.zeros([N-2,N-2]) # matriz que resolvera el sistema de ecuaciones, dimensiones N-2, N-2, cuadrada, tiene 2 nodos menos q la matriz de resultados Pimplicita


In [ ]:
for i in range(N-2):              # para Llenar valores de matriz de resultados Pimplicita se apoya de la matrz de resolucion. Pyton reconoce con indice 0 a frontera izq y der (primera y ult columna), en la matriz Pimplicita
  Matriz_resolucion[i,i]=1+2*Eta  # Matriz de resolucion sst ecuaciones.-seteos de valores de diagonal versus asignacion de indices [i,i], filas igual a columnas. En Pyton, primera fila matriz resolucion tiene indice cero
  if i>0:                          # Matriz de resolucion.-Pyton segunda fila, lado izquierdo de la diagonal i=1 (por debajo); el valor nuerico es -Eta
    Matriz_resolucion[i,i-1]=-Eta
  if i<(N-3):                     # Matriz de resolucion N-2 x N-2.- por encima de la diagonal  en N-3 (Pyton) el valor nuerico es -Eta (ver 2h15 min video)
    Matriz_resolucion[i,i+1]=-Eta


In [ ]:
# Resolucion de sistema ecuaciones matriciales en Pyton A.Pexp(n+1)= Poexp(n) ver video 2h30
